In [2]:
import tensorflow as tf
from keras import __version__
tf.keras.__version__ = __version__
from imp import reload

import time
import random
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Embedding, Reshape

from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

from env_class import BatteryManagementEnv

plt.style.use("ggplot")

In [3]:
env = BatteryManagementEnv()
env.reset()

array([ 1.000e+01,  2.832e+01,  1.007e+01, -4.080e+00, -9.910e+00,
       -7.410e+00, -1.255e+01, -1.725e+01, -1.507e+01, -4.930e+00,
       -6.330e+00, -4.930e+00,  4.500e-01,  1.200e-01, -2.000e-02,
        0.000e+00, -3.000e-02,  1.970e+00,  9.060e+00,  7.000e-02,
       -4.970e+00, -6.980e+00, -2.493e+01, -4.870e+00, -2.893e+01,
        0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
        2.500e+01,  2.500e+01,  2.500e+01,  2.500e+01,  1.000e+02,
        1.000e+02,  1.000e+02,  1.000e+02,  1.000e+02,  1.000e+02,
        5.000e+01,  5.000e+01,  5.000e+01,  5.000e+01,  0.000e+00,
        0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00])

In [4]:
print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

Action Space 1
State Space 49


In [5]:
print(env.nA)

5


In [6]:
from keras.layers import Dense, Input
from keras.models import Model

#build neural network for DQN
def build_model(states, actions):
    input = Input(shape=(1,states))
    x = Flatten()(input)
    x = Dense(16, activation='relu')(x)
    #output layer
    output = Dense(actions, activation='linear')(x)

    model = Model(inputs=input, outputs=output)

    return model

model = build_model(env.observation_space, env.nA)

In [7]:
from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy, EpsGreedyQPolicy, LinearAnnealedPolicy
from rl.memory import SequentialMemory
from keras.src.saving import serialization_lib
serialization_lib.enable_unsafe_deserialization()
from tensorflow.keras.optimizers.legacy import Adam

# Then, define DQN agent in Keras-RL
memory = SequentialMemory(limit=20000, window_length=1)
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(eps=0), 
                              attr='eps', value_max=1., value_min=.1, value_test=.05, nb_steps=10000)
dqn = DQNAgent(model=model, nb_actions=env.nA, memory=memory, policy=policy,
                nb_steps_warmup=500, target_model_update=1e-2, enable_double_dqn=True, enable_dueling_network=True)
dqn.compile(optimizer=Adam(learning_rate=1e-3), metrics=['mae'])

In [8]:
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1, nb_max_episode_steps=10, log_interval=1000)

Training for 50000 steps ...
Interval 1 (0 steps performed)


Day: 1
[-3.357e+01 -4.592e+01 -4.829e+01 -4.499e+01 -4.893e+01 -2.991e+01
 -1.000e-02  3.743e+01  4.806e+01  5.074e+01  4.757e+01  4.394e+01
  4.097e+01  4.495e+01  4.964e+01  5.367e+01  5.601e+01  5.695e+01
  6.208e+01  6.211e+01  5.799e+01  5.564e+01  5.513e+01  5.076e+01]
   1/1000 [..............................] - ETA: 2:15 - reward: 0.0000e+00Day: 2
[42.91 45.22 45.63 44.   43.88 45.92 51.07 52.77 62.89 60.03 58.19 62.99
 63.52 64.67 65.24 67.76 68.41 69.55 67.28 69.46 68.38 61.72 53.72 49.98]
Day: 3
[50.73 47.11 47.07 46.94 47.   46.91 49.59 55.32 55.78 55.52 55.23 53.58
 51.74 51.6  51.41 51.69 52.59 54.66 54.1  51.89 46.58 45.43 43.96 31.41]
Day: 4
[26.9  25.12 24.12 22.04 18.37 22.09 23.35 28.76 36.63 40.46 45.85 49.8
 51.36 51.74 51.92 53.22 56.62 56.92 61.64 59.44 52.75 51.9  51.38 49.96]
Day: 5
[50.29 47.72 48.38 48.02 44.23 47.17 48.19 49.11 50.44 53.4  56.55 60.02
 60.22 55.   52.39 51.57 54.71 63.43 67.37 67.

c:\Users\janle\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Day: 3
[50.73 47.11 47.07 46.94 47.   46.91 49.59 55.32 55.78 55.52 55.23 53.58
 51.74 51.6  51.41 51.69 52.59 54.66 54.1  51.89 46.58 45.43 43.96 31.41]
Day: 4
[26.9  25.12 24.12 22.04 18.37 22.09 23.35 28.76 36.63 40.46 45.85 49.8
 51.36 51.74 51.92 53.22 56.62 56.92 61.64 59.44 52.75 51.9  51.38 49.96]
Day: 5
[50.29 47.72 48.38 48.02 44.23 47.17 48.19 49.11 50.44 53.4  56.55 60.02
 60.22 55.   52.39 51.57 54.71 63.43 67.37 67.2  66.03 55.36 58.59 53.7 ]
Day: 6
[46.03 47.98 47.84 46.11 46.08 47.62 55.77 68.61 74.15 74.93 73.59 71.23
 68.79 66.75 62.47 53.25 53.26 53.42 47.91 42.05 40.96 32.04 20.82  1.84]
Day: 7
[17.94 20.91  7.78 14.33 18.56 18.57 35.81 43.87 46.93 43.88 43.85 46.74
 43.94 43.21 43.81 45.6  35.21 45.64 45.63 37.94 39.53 35.97 29.72 22.55]
Day: 8
[20.04  7.24  3.43 10.04 14.2  25.41 36.98 43.89 50.98 50.3  50.21 45.8
 43.64 43.49 45.05 49.95 53.82 55.94 54.19 54.19 52.98 51.68 48.95 47.63]
Day: 9
[46.48 49.08 47.11 48.8  48.5  49.8  56.97 72.6  81.17 81.79 81.15 80.
